In [1]:
import sqlite3
import pandas
import numpy

db = sqlite3.connect(r'products.db')
datadf = pandas.read_sql_query("SELECT itemid, timestamp, transactionid, productid  FROM items", db)
datadf.head()

,itemid,timestamp,transactionid,productid
0,1,2018-12-11 14:46:48,1,9
1,2,2018-12-11 14:46:49,1,10
2,3,2018-12-11 14:46:55,1,49
3,4,2018-12-11 14:49:25,2,3
4,5,2018-12-11 14:49:29,2,66


In [2]:
productsdf = pandas.read_sql_query("SELECT productid, name  FROM products", db)
productsdf.head()

,productid,name
0,1,Espresso
1,2,Americano
2,3,Latte
3,4,Cappuccino
4,5,Mocha


In [3]:

df = pandas.DataFrame(datadf['transactionid'], columns = ['transactionid'])
df['productid'] = datadf['productid']
data = df.values
grouped = df.groupby('transactionid')

itemIndex = [grouped['productid'].groups[i].values for i in datadf['transactionid']]


In [4]:
items = []
for j in range(len(grouped)): 
    a = [df['productid'][i]for i in range(itemIndex[j][0], itemIndex[j][0]+len(itemIndex[j]))]
    items.append(a)

In [5]:
def loadDataSet():
    return items



In [6]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))
    
    

In [7]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData



In [8]:
dataSet = loadDataSet()


In [9]:
C1 = createC1(dataSet)

In [10]:
C1


[frozenset({3}),
 frozenset({4}),
 frozenset({5}),
 frozenset({8}),
 frozenset({9}),
 frozenset({10}),
 frozenset({38}),
 frozenset({39}),
 frozenset({40}),
 frozenset({41}),
 frozenset({44}),
 frozenset({46}),
 frozenset({49}),
 frozenset({55}),
 frozenset({58}),
 frozenset({61}),
 frozenset({64}),
 frozenset({65}),
 frozenset({66}),
 frozenset({67}),
 frozenset({68}),
 frozenset({69}),
 frozenset({70}),
 frozenset({72})]

In [11]:
#D is a dataset in the setform.

D = list(map(set,dataSet))




In [12]:
L1,suppDat0 = scanD(D,C1,0.05)
L1

[frozenset({41}),
 frozenset({8}),
 frozenset({69}),
 frozenset({5}),
 frozenset({72}),
 frozenset({40}),
 frozenset({65}),
 frozenset({64}),
 frozenset({38}),
 frozenset({61}),
 frozenset({70}),
 frozenset({46}),
 frozenset({67}),
 frozenset({58}),
 frozenset({55}),
 frozenset({39}),
 frozenset({4}),
 frozenset({68}),
 frozenset({3}),
 frozenset({49}),
 frozenset({10}),
 frozenset({9})]

In [13]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList



In [14]:
def apriori(dataSet, minSupport = 0.1277):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData




In [15]:
L,suppData = apriori(dataSet)


In [16]:
L



[[frozenset({72}),
  frozenset({64}),
  frozenset({70}),
  frozenset({46}),
  frozenset({39}),
  frozenset({4}),
  frozenset({68}),
  frozenset({9})],
 [frozenset({39, 70})],
 []]

In [17]:
print('Most Frequent Items:\n')

for l in L[0]:
    print(productsdf['name'][[next(iter(l)) for _ in range(1)][0]-1])


Most Frequent Items:

Kettle Chips Lighly Salted
Sandwich Cheese and Onion
Kettle Chips Cheddar and Onion
Panini Goat's Cheese, Med. Veg
Coke Zero 330ml
Cappuccino
Triple Chocolate Cookie
Camomile Tea


In [18]:
def generateRules(L, supportData, minConf=0.9):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [19]:
def calcConf(freqSet, H, supportData, brl, minConf=0.9):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [20]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.9):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            
            
            

In [21]:
L,suppData= apriori(dataSet,minSupport=0.1277)



In [22]:
rules= generateRules(L,suppData, minConf=0.9)



frozenset({70}) --> frozenset({39}) conf: 1.0


In [23]:
prod1= rules[0][0]
prod1 = [next(iter(prod1)) for _ in range(1)]

prod2 = rules[0][1]
prod2 = [next(iter(prod2)) for _ in range(1)]

conf = rules[0][2]

print('Rules:')
print(productsdf['name'][prod1[0]-1],' --> ',productsdf['name'][prod2[0]-1], ' with conf: ', conf)


Rules:
Kettle Chips Cheddar and Onion  -->  Coke Zero 330ml  with conf:  1.0
